In [3]:
# Import necessary libraries
import os
import random
import numpy as np
import pandas as pd
import kagglehub # Import kagglehub

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from datasets import Dataset

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)

import torch

# 1. Load dataset using kagglehub
dataset_path = kagglehub.dataset_download('lakshmi25npathi/imdb-dataset-of-50k-movie-reviews')
csv_file_path = os.path.join(dataset_path, 'IMDB Dataset.csv')
df = pd.read_csv(csv_file_path)

# Map sentiment to 0 (negative), 1 (positive)
df['label'] = df['sentiment'].map({'negative': 0, 'positive': 1})

# 2. Split train-test (80-20 split)
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)
# 3. Convert to Huggingface Datasets
train_dataset = Dataset.from_pandas(train_df[['review', 'label']])
test_dataset = Dataset.from_pandas(test_df[['review', 'label']])

# 4. Use smaller subset for quick experiments (first 4000 train, 1000 test)
small_train = train_dataset.shuffle(seed=42).select(range(4000))
small_test = test_dataset.shuffle(seed=42).select(range(1000))

# 5. Models to test
model_names = [
    "distilbert-base-uncased",
    "bert-base-uncased",
    "roberta-base",
    "microsoft/deberta-v3-base",
    "google/electra-base-discriminator"
]
# 6. Tokenizer helper
def tokenize(batch, tokenizer):
    return tokenizer(batch["review"], padding="max_length", truncation=True, max_length=256)

# 7. Custom F1 metric
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, preds, average="weighted")
    return {"f1": f1}

# 8. Training loop to find best model
best_f1 = 0
best_model_name = None
best_trainer = None

for model_name in model_names:
    print(f"\n\n===== Training {model_name} on subset =====")
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    tokenized_train = small_train.map(lambda x: tokenize(x, tokenizer), batched=True)
    tokenized_test = small_test.map(lambda x: tokenize(x, tokenizer), batched=True)

    tokenized_train.set_format("torch", columns=["input_ids", "attention_mask", "label"])
    tokenized_test.set_format("torch", columns=["input_ids", "attention_mask", "label"])

    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

    training_args = TrainingArguments(
        output_dir=f"./results-{model_name.replace('/', '-')}",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=32,
        num_train_epochs=3,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        greater_is_better=True,
        save_total_limit=1,
        seed=42,
        logging_dir=f"./logs-{model_name.replace('/', '-')}",
        logging_strategy="epoch",
        eval_strategy="epoch",     # Corrected parameter name for Transformers 4.x
        save_strategy="epoch",
        disable_tqdm=False,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_test,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
    )

    trainer.train()
    eval_results = trainer.evaluate()
    f1_score_val = eval_results['eval_f1']
    print(f"Model {model_name} F1 score on subset test set: {f1_score_val:.4f}")

    if f1_score_val > best_f1:
        best_f1 = f1_score_val
        best_model_name = model_name
        best_trainer = trainer

print(f"\nBest model on subset: {best_model_name} with F1: {best_f1:.4f}")
# 9. Fine-tune best model on full train and test sets
print(f"\n\n===== Fine-tuning best model ({best_model_name}) on full dataset =====")
tokenizer = AutoTokenizer.from_pretrained(best_model_name)
tokenized_train_full = train_dataset.map(lambda x: tokenize(x, tokenizer), batched=True)
tokenized_test_full = test_dataset.map(lambda x: tokenize(x, tokenizer), batched=True)
tokenized_train_full.set_format("torch", columns=["input_ids", "attention_mask", "label"])
tokenized_test_full.set_format("torch", columns=["input_ids", "attention_mask", "label"])

model = AutoModelForSequenceClassification.from_pretrained(best_model_name, num_labels=2)

training_args_full = TrainingArguments(
    output_dir=f"./results-full-{best_model_name.replace('/', '-')}",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    save_total_limit=1,
    seed=42,
    logging_dir=f"./logs-full-{best_model_name.replace('/', '-')}",
    logging_steps=100,
    eval_strategy="epoch", # Corrected parameter name for Transformers 4.x
    save_strategy="epoch",
)

trainer_full = Trainer(
    model=model,
    args=training_args_full,
    train_dataset=tokenized_train_full,
    eval_dataset=tokenized_test_full,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

trainer_full.train()
eval_results_full = trainer_full.evaluate()
print(f"Final model F1 score on full test set: {eval_results_full['eval_f1']:.4f}")
# 10. Inference on 10 random test reviews
print("\nSample Inference on 10 random test reviews:\n")
sample_indices = random.sample(range(len(test_dataset)), 10)
for idx in sample_indices:
    text = test_dataset[idx]['review']
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256)
    inputs = {k: v.to(trainer_full.model.device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = trainer_full.model(**inputs)
    pred = torch.argmax(outputs.logits, dim=1).item()
    label = "Positive" if pred == 1 else "Negative"
    print(f"Review: {text[:150].replace(chr(10), ' ')}...")
    print(f"Prediction: {label}\n")



===== Training distilbert-base-uncased on subset =====


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-208986528.py:145: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_full = Trainer(


Epoch,Training Loss,Validation Loss,F1
1,0.223100,0.194800,0.934264
2,0.156900,0.248253,0.938796
3,0.100300,0.265867,0.943796


Final model F1 score on full test set: 0.9438

Sample Inference on 10 random test reviews:

Review: I am Curious (Yellow) (a film, in near Seussical rhyme, is said right at the start to be available in two versions, Yellow and Blue) was one of those ...
Prediction: Positive

Review: I, like most other people, saw this movie after hearing of it from Patton Oswalt. Oddly enough, it was easier to find than I thought it would be. Thou...
Prediction: Negative

Review: Any one who saw the original would have to go out and destroy this dreadful remake. Alex Baldwin trying to imitate the late Steve Mcqueen in a word fo...
Prediction: Negative

Review: Square pigs as a way of efficiently utilizing cargo space is the one and only clever moment in "Space Truckers". Believe it or not, Dennis Hopper has ...
Prediction: Negative

Review: Mona the vagabond lives on the fringes of French society, in a life without meaning, purpose or direction.<br /><br />I watched this because of all th...
Prediction